In [11]:
from vqf.preprocessing import create_clauses
from itertools import product
import pandas as pd
from IPython.display import display
from sympy import simplify

import sys

sys.path.append(f"./../")
from src.clause_utils import table_form

In [12]:
# p = 5
# q = 3

# p = 7
# q = 3

# p = 11
# q = 7

# p = 241
# q = 233

p = 13
q = 11

# p = 557
# q = 523

m = p * q

p_bits, q_bits, z_bits, clauses = create_clauses(
    m, apply_preprocessing=False, verbose=False
)

for c in clauses:
    display(c)

p_0*q_0 - 1

p_0*q_1 + p_1*q_0 - 2*z_1_2 - 4*z_1_3 - 8*z_1_4 - 16*z_1_5 - 32*z_1_6 - 64*z_1_7 - 1

p_0*q_2 + p_1*q_1 + p_2*q_0 + z_1_2 - 2*z_2_3 - 4*z_2_4 - 8*z_2_5 - 16*z_2_6 - 32*z_2_7 - 1

p_0*q_3 + p_1*q_2 + p_2*q_1 + p_3*q_0 + z_1_3 + z_2_3 - 2*z_3_4 - 4*z_3_5 - 8*z_3_6 - 16*z_3_7 - 1

p_1*q_3 + p_2*q_2 + p_3*q_1 + p_4*q_0 + z_1_4 + z_2_4 + z_3_4 - 2*z_4_5 - 4*z_4_6 - 8*z_4_7

...

p_3*q_3 + p_4*q_2 + p_5*q_1 + p_6*q_0 + z_1_6 + z_2_6 + z_3_6 + z_4_6 + z_5_6 - 2*z_6_7

p_4*q_3 + p_5*q_2 + p_6*q_1 + p_7*q_0 + z_1_7 + z_2_7 + z_3_7 + z_4_7 + z_5_7 + z_6_7 - 1

p_5*q_3 + p_6*q_2 + p_7*q_1

p_6*q_3 + p_7*q_2

p_7*q_3

In [13]:
p_bits_simple, q_bits_simple, z_bits_simple, simplified_clauses = create_clauses(
    m, apply_preprocessing=True, verbose=False
)

for c in simplified_clauses:
    display(c)

0

0

0

p_3 - 2*q_1*q_2 + q_1 + q_2 + q_3 - 2*z_3_4 - 1

p_3*q_1 + p_4 - q_1*q_3 + q_3 + z_3_4 - 2*z_4_5 - 4*z_4_6

...

p_3*q_3 + p_4*q_2 + p_5*q_1 + p_6 + z_4_6 + z_5_6 - 2*z_6_7

p_4*q_3 + p_5*q_2 + p_6*q_1 + p_7 + z_5_7 + z_6_7 - 1

p_5*q_3 + p_6*q_2 + p_7*q_1

p_6*q_3 + p_7*q_2

p_7*q_3

In [14]:
def max_disjoint_sets(sets):
    """
    Finds the maximum number of disjoint sets from a given list of sets.

    Args:
    -----

        `sets` (`list`): A list of sets.

    Returns:
    --------

        `tuple`: A tuple containing the disjoint sets themselves, and their indices in the original list.

    Raises:
    --------

        `ValueError`: If the maximum number of disjoint sets is 1, indicating that no disjoint sets were found.
    """

    # Sort the list of sets in increasing order of their sizes
    sorted_set = sorted(sets, key=len, reverse=False)

    # Initialize an empty list to store the disjoint sets
    disjoint_sets = []

    # Iterate over the sorted list of sets
    for i, s in enumerate(sorted_set):
        # Check if the current set intersects with any of the sets in the disjoint sets list
        if not any(s.intersection(ds) for ds in disjoint_sets):
            # If not, add it to the disjoint sets list
            if s:
                disjoint_sets.append(s)

    max_num_sets = len(disjoint_sets)

    if max_num_sets == 1:
        raise ValueError("No disjoint sets found. Choose another set of sets.")

    # Get the indices of the disjoint sets in the original list
    indices = [sets.index(ds) for ds in disjoint_sets]

    # Return the disjoint sets along with their indices in the original list
    return disjoint_sets, indices


def get_disjoint_clauses(simplified_clauses):
    disjoint_clauses = []
    var_sets = [c.free_symbols for c in simplified_clauses]
    disjoint_sets, indices = max_disjoint_sets(var_sets)

    for i in indices:
        disjoint_clauses.append(simplified_clauses[i])

    return disjoint_clauses


disjoint_clauses = get_disjoint_clauses(simplified_clauses)

for c in disjoint_clauses:
    display(c)

ValueError: No disjoint sets found. Choose another set of sets.

In [ ]:
def reduce_space(expr):
    variables = list(expr.free_symbols)
    values = product(range(2), repeat=len(variables))
    reduced_space = []
    for v in values:
        subs_dict = dict(zip(variables, v))
        subs_expr = expr.subs(subs_dict)
        if simplify(subs_expr) == 0:
            reduced_space.append(v)
    return variables, reduced_space


frames = []
for c in disjoint_clauses:
    variables, reduced_space = reduce_space(c)
    df = pd.DataFrame(reduced_space, columns=variables)
    frames.append(df)

for df in frames:
    display(df)

,p_15,q_7
0,0,0
1,0,1
2,1,0


,q_2,z_2_3,z_2_4,q_1
0,0,0,0,0
1,0,1,0,1
2,1,0,1,1
3,1,1,0,0


In [ ]:
def process_dataframe(df):
    # Remove columns starting with "z_"
    df = df.filter(regex=r"^(?!z_)")

    # Remove duplicate rows
    df = df.drop_duplicates()

    return df


super_bits = set()
for df in frames:
    df = process_dataframe(df)
    super_bits |= set(df.columns)
    display(df)

,p_15,q_7
0,0,0
1,0,1
2,1,0


,q_2,q_1
0,0,0
1,0,1
2,1,1
3,1,0


In [ ]:
total_bits = len(p_bits) + len(q_bits)
print(f"The total number of bits (excludig z-terms): {total_bits}")
print(f"The number of superposition-bits: {len(super_bits)}")

The total number of bits (excludig z-terms): 24
The number of superposition-bits: 4


In [ ]:
def create_merged_dict(dict1, dict2):
    """
    Create a new dictionary by merging values from two dictionaries.

    The function takes two dictionaries as input and returns a new dictionary
    where the keys are the values from the first dictionary and the values are
    the corresponding values from the second dictionary.

    Args:
    -----

        `dict1` (`dict`): The first dictionary.
        `dict2` (`dict`): The second dictionary.

    Returns:
    --------

        `dict`: A new dictionary with merged values.

    Examples:
    ---------

        >>> dict1 = {'apple': 'fruit', 'carrot': 'vegetable', 'banana': 'fruit'}
        >>> dict2 = {'fruit': 'red', 'vegetable': 'orange'}
        >>> merged_dict = create_merged_dictionary(dict1, dict2)
        >>> merged_dict
        {'fruit': 'red', 'vegetable': 'orange'}
    """
    merged_dict = {}
    for key, value in dict1.items():
        merged_dict[value] = dict2[key]
    return merged_dict


p_bits = create_merged_dict(p_bits, p_bits_simple)
q_bits = create_merged_dict(q_bits, q_bits_simple)

In [ ]:
table_form(p_bits)

Key    Value
-----  -------
p_0    1
p_1    q_1
p_2    q_2
p_3    p_3
p_4    p_4
p_5    p_5
p_6    p_6
p_7    p_7
p_8    p_8
p_9    p_9
p_10   p_10
p_11   p_11
p_12   p_12
p_13   p_13
p_14   p_14
p_15   p_15


In [ ]:
table_form(q_bits)

Key    Value
-----  -------
q_0    1
q_1    q_1
q_2    q_2
q_3    q_3
q_4    q_4
q_5    q_5
q_6    q_6
q_7    q_7
